# Fine-tuning du modèle GPT-2 sur des articles scientifiques

Dans ce notebook, nous nous sommes focalisés sur l'entrainement par fine-tuning du modèle GPT-2, sur un jeu de données d'articles scientifiques scientific_papers (dataset Tensorflow).

L'objectif est de pouvoir spécialiser un modèle générique (modèle GPT-2 base) sur un domaine spécifique via un entrainement sur un jeu de données fourni (dataset TensorFlow scientific_papers)

**Groupe ALOQAS**
- Aurélien ZUFIC
- Lucas AGUETAÏ
- Ony ANDRIATSAHAVOJAONA
- Quentin VERMEERSCH
- Alexandre HUYNH
- Samuel DORISMOND

**NOTE /!\ : L'entrainement s'est effectué en deux temps (TRAIN PART 1 & 2), étant donné qu'il était impossible de charger et entraîner le modèle sur le jeu de données en entier (limitations mémoire RAM). Il sera nécessaire de commenter et décommenter certaines sections de code pour l'exécution en cas d'entrainement à partir du début.**

---
# Import de packages
Importation des packages Python nécessaires pour le fine-tuning.

Nous chargeons également un package personnel datasets_scientific_paper, contenant des fonctions Python écrites pour le chargement de la dataset localement.

Celle-ci est divisée en plusieurs parties à travers plusieurs fichiers, afin de prendre en compte les performances limités de Google Colaboratory.

In [ ]:
!pip install keras_nlp -q
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
import os
import gc
import tensorflow as tf
from tensorflow import keras
from datasets import load_dataset
import json
import keras_nlp
import time
import numpy as np

Using TensorFlow backend


---
# Chargement dataset train en local

Pour le fine-tuning du modèle GPT-2, nous utilisons pour l'instant seulement les articles d'entrainement (le jeu de données entier d'origine est divisée en 3 parties : train, val, test).

Le package importée précédemment nous permet d'utiliser une fonction personnalisée load_dataset pour charger dans des variables Python le jeu de données sous forme de dictionnaires et de listes.

In [ ]:
dataset = load_dataset("scientific_papers", 'pubmed')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scientific_papers
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warn

Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

'\ntrain_dataset = dataset["train"]\ntest_dataset = dataset["test"]\nval_dataset = dataset["validation"]\n'

---
# Initialisation d'un modèle GPT-2 de base
Nous initialisons une instance du modèle GPT-2 sur lequel nous effectuons l'entrainement.

Le modèle et code utilisés sont celui proposés via Keras NLP.

Il existe plusieurs modèles préentrainées pour GPT-2, ayant un nombre de couches et de paramètres différent : celui sélectionné dans notre cas est le modèle gpt2_base_en, qui est suffisant pour notre entrainement.

In [ ]:
os.environ["KERAS_BACKEND"] = "tensorflow"  # or "tensorflow" or "torch"

keras.mixed_precision.set_global_policy("mixed_float16")

preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=128,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

100%|██████████| 448/448 [00:00<00:00, 293kB/s]
100%|██████████| 446k/446k [00:00<00:00, 1.46MB/s]
100%|██████████| 0.99M/0.99M [00:00<00:00, 2.74MB/s]
100%|██████████| 484/484 [00:00<00:00, 1.13MB/s]
100%|██████████| 475M/475M [00:08<00:00, 55.8MB/s]
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


Nous définissons une fonction articles_to_text nous permettant de concaténer tous les mots des articles dans une liste Python unique.

En effet, chaque article possèdent plusieurs champs d'attributs : article_text, abstract, section_names...

Le plus important pour le fine-tuning est d'entrainer le modèle sur le contenu textuel, donc article_text. Cette dernière - après chargement via notre fonction personnalisée - est composée pour chaque article d'une liste Python constitué de sous listes python pour plusieurs portions du texte. La fonction présente reformatte cela afin de ne garder qu'une seule liste Python contenant tous les mots sur lequel l'apprentissage doit se faire.

---
# Chargement des paramètres d'entrainement

Il existe plusieurs méthodes pour l'enregistrement des résultats de l'entrainement d'un modèle sur un support fichier.

Basé sur les enseignements fournis par notre enseignant M. Faye, nous avons retenus trois méthodes :
- **Fichier .keras** : enregistrement du **modèle** (architecture, couches)
- **Fichier .ckpt** : enregistrement des **paramètres / poids** mobilisés
- **Fichier .h5** : enregistrement du **modèle** et des **poids** mobilisés

Il nous a été recommandé pour le fine-tuning de préférer l'enregistrement des paramètres. Il nous suffit donc à chaque début d'initialiser le modèle GPT-2 de base et de charger l'entrainement effectué via le chargement des poids/paramètres (fonction .load_weights).

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/training_data_all_remake_epochs/cp.ckpt"

Mounted at /content/drive


In [ ]:
# Chargement des poids du modèle (facultatif)
# - exécuter seulement si des fichiers de checkpoints existent
gpt2_lm.load_weights(checkpoint_path)

/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:47: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:47: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids


---
# Entrainement / Fine-tuning

C'est dans cette partie que le fine-tuning du modèle GPT-2 est réalisé.

## Principe

Le principe de ce fine-tuning est de fournir un grand nombre de mots et de phrases d'articles scientifiques dans un tableau Python qui sera fourni à une fonction .fit pour l'entrainement du modèle. Le modèle va se charger d'apprendre sur ce qui est fourni afin d'orienter la génération de texte du modèle GPT-2 vers un vocabulaire et style similaire aux articles scientifiques.

Dans ce code, si des fichiers de checkpoint sont trouvés, le programme charge les poids existants, sinon il entraine le modèle et sauvegarde de nouveaux poids.

## Entrainement sur échantillons d'articles scientifiques

Nous fournissons à la fonction .fit réalisant l'entrainement une liste Python features, auquel on donne comme liste le contenu textuel d'articles obtenus via la fonction articles_to_text, mentionnée précédemment.

Plusieurs essais ont été menés progressivement avec des échantillons de nombre croissant. Pour s'assurer que l'entrainement était faisable compte tenu des performances et ressources à notre disposition, nous avions débuté avec 1 à 10 articles.

Nous avons procédé par la suite à des nombres d'articles plus importants : 10.000 articles, 50.000, et enfin 80.000 articles, limite atteinte d'articles mobilisables dans un même entrainement.

## Callback ModelCheckpoint

Un callback est utilisé afin d'effectuer une sauvegarde des paramètres à chaque "checkpoint" (point de contrôle) en fin d'epoch. Etant donné que nous avons défini seulement 1 seul epoch, l'enregistrement des poids se fait en fin d'entrainement. <br />
Cela entraîne la création de fichiers .ckpt constituant l'entrainement du modèle, sous la forme de divers poids pour le modèle GPT-2.

## Mobilisation du GPU

Afin de permettre la réalisation de l'entrainement en évitant la saturation de la mémoire vive, il était nécessaire de mobiliser le GPU en plus de la RAM.

Nous avons donc inclus notre dans une structure Python spécifique qui nous permet de solliciter le GPU pour une partie spécifique du code.
```python
with tf.device('/device:GPU:0'):
```

In [ ]:
# Définir le chemin et le répertoire du checkpoint
checkpoint_dir = os.path.dirname(checkpoint_path)

# Créer le répertoire s'il n'existe pas
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# features = dataset["train"][:59963]["article"] # fait
features = dataset["train"][59963:]["article"]

# Créer un callback pour sauvegarder les poids du modèle
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

print("Training model...")
# Entraîner le modèle avec GPU et utiliser le callback de checkpoint
with tf.device('/device:GPU:0'):
    num_epochs = 3

    # Taux d'apprentissage décroissant linéairement
    learning_rate = keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=5e-5,
        decay_steps=len(features) * num_epochs,
        end_learning_rate=0.0,
    )

    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    gpt2_lm.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=loss,
        weighted_metrics=["accuracy"],
    )

    # Entraîner le modèle avec le callback de checkpoint
    gpt2_lm.fit(
        x=features,
        epochs=num_epochs,
        callbacks=[cp_callback]  # Passer le callback de checkpoint à l'entraînement
    )

Training model from scratch...
Epoch 1/3
1874/1874 [==============================] - ETA: 0s - loss: 3.1154 - accuracy: 0.4102
Epoch 1: saving model to /content/drive/MyDrive/Colab Notebooks/training_data_all_remake_epochs/cp.ckpt


/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:47: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/task.py:47: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return id(getattr(self, attr)) not in self._functional_layer_ids
/usr/local/lib/python3.10/dist-packages/keras_nlp/src/models/backbone.py:37: UserWarning: 

1874/1874 [==============================] - 2099s 1s/step - loss: 3.1154 - accuracy: 0.4102
Epoch 2/3
1874/1874 [==============================] - ETA: 0s - loss: 2.9059 - accuracy: 0.4348
Epoch 2: saving model to /content/drive/MyDrive/Colab Notebooks/training_data_all_remake_epochs/cp.ckpt
1874/1874 [==============================] - 1805s 962ms/step - loss: 2.9059 - accuracy: 0.4348
Epoch 3/3
1874/1874 [==============================] - ETA: 0s - loss: 2.8099 - accuracy: 0.4457
Epoch 3: saving model to /content/drive/MyDrive/Colab Notebooks/training_data_all_remake_epochs/cp.ckpt
1874/1874 [==============================] - 1817s 969ms/step - loss: 2.8099 - accuracy: 0.4457


In [ ]:
# Rassembler le texte des articles en une seule liste
val_features = articles_to_text(val_dataset, "article")

In [ ]:
# Évaluer le modèle sur les données d'évaluation
with tf.device('/device:GPU:0'):
  val_loss, val_accuracy = gpt2_lm.evaluate(
      x=val_features,
      verbose=1
  )

# Afficher les résultats de l'évaluation
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

In [ ]:
# Rassembler le texte des articles en une seule liste
test_features = articles_to_text(test_dataset, "article")

In [ ]:
# Évaluer le modèle sur les données de test
with tf.device('/device:GPU:0'):
  test_loss, test_accuracy = gpt2_lm.evaluate(
      x=test_features,
      verbose=1
  )

# Afficher les résultats de l'évaluation
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

---
# Test génération de texte à partir de prompt

Nous avons essayé de générer du texte à partir d'exemple de prompts, afin de vérifier que le modèle reste cohérent après celui, et également pour avoir un aperçu de la spécialisation de celle-ci sur les thèmatiques des articles scientifiques.

In [ ]:
# Exemple de prompts basés sur des sujets de papiers scientifiques
prompts = [
    "The impact of global warming on marine biodiversity",
    "Technological advancements in renewable energy sources",
    "Genetic factors influencing Alzheimer's disease",
    "The role of artificial intelligence in personalized medicine",
    "Quantum computing and its future implications",
    "Mechanisms of resistance to antibiotics in bacteria",
    ""
]

# Parcourir et générer des réponses pour chaque prompt
for prompt in prompts:
    start = time.time()
    output = gpt2_lm.generate(prompt, max_length=200)
    end = time.time()

    print(f"\nPrompt: {prompt}")
    print("GPT-2 output:")
    print(output)
    print(f"TOTAL TIME ELAPSED: {end - start:.2f}s")
    print('________________________________________________')